# 示例
### 任务链接：https://www.yuque.com/ol1q37/gi94xp/vy59wr
### 参考链接：https://pintia.cn/problem-sets/1111652100718116864/problems/type/6
###           https://pintia.cn/problem-sets/1111652100718116864/problems/type/7
### 1）存储所有要采集页面的完整html
### 2）编写代码完成数据的抽取
### 3）设计数据库表结构（包括标号、标题、分数、通过数、提交数、通过率、题目内容、完整html、题型（编程题 、函数题），题目详情页链接）
### 4）存储到sqlite数据库中

## 解题思路：分别编写获取列表页和详情页的函数，最后存储到数据库中

In [ ]:
import re,time
from selenium import webdriver

def extract_list_info(url,category):
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(5)#缓冲5秒
    html = driver.page_source#获取源代码
    result = []
    reobj = re.compile(r'<tr><td class="answerIcon_1du7d"></td><td>([\d\D]*?)</td><td><a href="([\d\D]*?)" class="">([\d\D]*?)</a></td><td>([\d\D]*?)</td><td>([\d\D]*?)</td><td>([\d\D]*?)</td><td>([\d\D]*?)</td></tr>')
    for match in reobj.finditer(html):#获取列表页的标号、链接、标题、分数、通过数、提交数、通过率
        biaohao = match.group(1)
        lianjie = "https://pintia.cn" + match.group(2)
        biaoti = match.group(3)
        fenshu = match.group(4)
        tongguoshu = match.group(5)
        tijiaoshu = match.group(6)
        tongguolv = match.group(7)
        result.append((category,biaohao,lianjie,biaoti,fenshu,tongguoshu,tijiaoshu,tongguolv))
    return result#返回元组列表

def extract_detail_info(results):
    list1 = []
    final_result = []
    for i in results:
        url1=i[2]#从中获取出链接
        driver=webdriver.Chrome()
        driver.get(url1)
        time.sleep(5)#缓冲5秒
        html1=driver.page_source#获取网页源代码
        reobj = re.compile(r'<div class="text-center black-3 text-4 font-weight-bold my-3">([\d\D]*?)<div class="problemInfo_OZ-Rv"><div class="d-flex flex-none align-items-center item_5Oc1c"><div class="label_13jiU">')
        for match in reobj.finditer(html1):
            list1.append(match.group(1))
    for x,y in zip(results,list1):#将详情页的内容与列表页内容拼接
        final_result.append((x,y))
    return final_result#返回元组列表

final_results = []
urls = [("https://pintia.cn/problem-sets/1111652100718116864/problems/type/6","函数题"),("https://pintia.cn/problem-sets/1111652100718116864/problems/type/7","编程题")]
for url,category in urls:   
    results = extract_list_info(url,category)#获取列表页的内容
    results = extract_detail_info(results)#获取详情页面的内容,并且与列表信息合并
    final_result.extend(results)

from pony.orm import *


db = Database()   #创建数据库对象


class Ttitle(db.Entity):
    lx = Optional(str)
    bh = PrimaryKey(str, auto=True)
    url = Optional(str)
    bt = Optional(str)
    fs = Optional(str)
    tgs = Optional(str)
    tjs = Optional(str)
    tgl = Optional(str)
    sj = Optional(str)



#db.generate_mapping()
db.bind(provider='sqlite', filename='D:/code2020/program.sqlite')#provider是数据库的类型,filename是文件的路径

db.generate_mapping(create_tables=True)
#将实体映射到数据库表,参数create_tables = True表示如果实体指向的表尚不存在，则使用CREATE TABLE命令创建它们。必须在调用generate_mapping（）方法之前定义连接到数据库的所有实体。

set_sql_debug(True)
#调试模式,使用set_sql_debug（）函数，您可以看到Pony发送到数据库的SQL命令。
for x in final_results:#循环写入到program.sqlite文件中去
    s = Ttitle(lx=x[0],bh=x[1],url=x[2],bt=x[3],fs=x[4],tgs=x[5],tjs=x[6],tgl=x[7],sj=x[8])
    db.commit()
    #Pony不会立即在数据库中保存对象。只有在调用commit()函数后才会保存这些对象。如果打开调试模式，那么在commit（）期间，您将看到发送到数据库的五个insert命令。